In [ ]:
# Progress tracking setup
import time
import json
import torch
from tqdm.notebook import tqdm

def track_time(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Operation completed in {end - start:.2f} seconds")
        return result
    return wrapper

In [ ]:
%%time
# Verify GPU availability and requirements
!nvidia-smi

import torch
gpu_name = torch.cuda.get_device_name(0)
if 'A100' not in gpu_name:
    print("⚠️ Warning: This model requires an A100 GPU for optimal performance")
    print("Current GPU:", gpu_name)

In [ ]:
%%time
# Package installation
%pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 \
    transformers==4.34.0 datasets accelerate huggingface_hub wandb -q

In [ ]:
%%time
from huggingface_hub import login, create_repo
from getpass import getpass
import wandb

# Get token securely
hf_token = getpass("Enter your Hugging Face access token: ")
login(token=hf_token)
print("Successfully logged in to Hugging Face!")

# Initialize W&B for experiment tracking
wandb.login()
print("Successfully logged in to Weights & Biases!")

In [ ]:
%%time
# Repository setup
!git clone https://github.com/VishwamAI/VishwamAI.git
%cd VishwamAI
%pip install -e . -q

In [ ]:
%%time
import torch
import json
from datasets import load_dataset, concatenate_datasets
from vishwamai.model_utils import load_model, get_gpu_memory
from vishwamai.model import Transformer, ModelArgs
from vishwamai.cache_augmentation import CacheConfig, DifferentiableCacheAugmentation
from vishwamai.neural_memory import ReasoningMemoryTransformer
from vishwamai.tree_of_thoughts import TreeOfThoughts
from vishwamai.reward_function import RewardConfig
from vishwamai.trainer import VishwamAIPretrainer

# Performance optimizations
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
@track_time
def setup_hardware():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = get_gpu_memory()
    print(f"Using GPU: {gpu_name} ({gpu_memory:.1f} GB)")

    # Optimize for available GPU
    if 'a100' in gpu_name.lower():
        variant = "671B"  # Full model
    elif 'v100' in gpu_name.lower():
        variant = "335B"  # Reduced size
    else:
        variant = "167B"  # Minimal configuration
    return variant

model_variant = setup_hardware()

In [ ]:
@track_time
def load_config():
    config_path = "./vishwamai/configs/config_optimized.json"
    with open(config_path) as f:
        config = json.load(f)
    
    if model_variant not in config["model_variants"]:
        raise KeyError(f"Model variant '{model_variant}' not found in config")
    
    return config["model_variants"][model_variant]["model_config"]

# Load configuration
model_config = load_config()
print("Configuration loaded successfully.")

In [ ]:
@track_time
def initialize_components():
    print("Initializing model and components...")
    
    # Initialize main model
    model_args = ModelArgs(
        max_batch_size=model_config["max_batch_size"],
        max_seq_len=model_config["max_seq_len"],
        dtype=model_config["dtype"],
        vocab_size=model_config["vocab_size"],
        dim=model_config["dim"],
        inter_dim=model_config["inter_dim"],
        moe_inter_dim=model_config["moe_inter_dim"],
        n_layers=model_config["n_layers"],
        n_dense_layers=model_config["n_dense_layers"],
        n_heads=model_config["n_heads"],
        n_routed_experts=model_config["n_routed_experts"],
        n_shared_experts=model_config["n_shared_experts"],
        n_activated_experts=model_config["n_activated_experts"],
        n_expert_groups=model_config["n_expert_groups"],
        n_limited_groups=model_config["n_limited_groups"],
        score_func=model_config["score_func"],
        route_scale=model_config["route_scale"],
        q_lora_rank=model_config["q_lora_rank"],
        kv_lora_rank=model_config["kv_lora_rank"],
        qk_nope_head_dim=model_config["qk_nope_head_dim"],
        qk_rope_head_dim=model_config["qk_rope_head_dim"],
        v_head_dim=model_config["v_head_dim"],
        original_seq_len=model_config["original_seq_len"],
        rope_theta=model_config["rope_theta"],
        rope_factor=model_config["rope_factor"],
        beta_fast=model_config["beta_fast"],
        beta_slow=model_config["beta_slow"],
        mscale=model_config["mscale"]
    )
    
    model = Transformer(model_args).cuda()
    
    # Initialize cache augmentation
    cache_config = CacheConfig(
        hidden_size=model_config["dim"],
        num_heads=model_config["n_heads"],
        max_cache_length=65536,
        dropout=0.1
    )
    cache_module = DifferentiableCacheAugmentation(cache_config).cuda()
    
    # Initialize memory transformer
    memory_module = ReasoningMemoryTransformer(
        hidden_size=model_config["dim"],
        num_heads=model_config["n_heads"]
    ).cuda()
    
    # Initialize tree of thoughts
    tree_module = TreeOfThoughts(
        hidden_size=model_config["dim"],
        num_heads=model_config["n_heads"]
    ).cuda()
    
    # Initialize reward config
    reward_config = RewardConfig(
        hidden_size=model_config["dim"],
        num_heads=model_config["n_heads"]
    )
    
    return model, cache_module, memory_module, tree_module, reward_config

model, cache_module, memory_module, tree_module, reward_config = initialize_components()

print(f"\nModel size: {sum(p.numel() for p in model.parameters())/1e9:.1f}B parameters")
print(f"Sequence length: {model_config['max_seq_len']:,} tokens")
print(f"Number of experts: {model_config['n_routed_experts']} routed + {model_config['n_shared_experts']} shared")
print(f"Active experts per token: {model_config['n_activated_experts']}")

In [ ]:
from transformers import TrainingArguments

# Initialize output directory
output_dir = "./pretrain_output"
!mkdir -p $output_dir

# Configure training
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=model_config['max_batch_size'],
    gradient_accumulation_steps=8,  # Adjust based on GPU memory
    learning_rate=1e-4,
    weight_decay=0.1,
    warmup_steps=1000,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    # Mixed precision training
    bf16=True,  # Use bfloat16
    fp16=False,
    # Performance optimizations
    gradient_checkpointing=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    group_by_length=True,
    # Monitoring
    report_to=["tensorboard", "wandb"],
    # Hub integration
    push_to_hub=True,
    hub_model_id="kasinadhsarma/vishwamai-model",
    hub_strategy="every_save",
    # Optimizer settings
    lr_scheduler_type="cosine",
    optim="adamw_torch",
    max_grad_norm=1.0,
    # Other settings
    remove_unused_columns=False,
    seed=42,
    ddp_find_unused_parameters=False
)

In [ ]:
from datasets import concatenate_datasets

# Load and combine training datasets
train_datasets = []
for ds_name in ["gsm8k", "cais/mmlu"]:
    try:
        dataset = load_dataset(ds_name, split="train")
        train_datasets.append(dataset)
    except Exception as e:
        print(f"Failed to load {ds_name}: {e}")

if not train_datasets:
    raise ValueError("No training datasets could be loaded")

combined_train_dataset = concatenate_datasets(train_datasets)

# Load validation dataset
try:
    eval_dataset = load_dataset("cais/mmlu", split="validation")
except Exception as e:
    print(f"Failed to load validation dataset: {e}")
    eval_dataset = None

In [ ]:
# Initialize trainer with all components
trainer = VishwamAIPretrainer(
    model=model,
    args=training_args,
    train_dataset=combined_train_dataset,
    eval_dataset=eval_dataset,
    memory_module=memory_module,
    tree_module=tree_module,
    cache_module=cache_module,
    reward_config=reward_config
)

# Start training
print("Starting training...")
start_time = time.time()
trainer.train()
training_time = time.time() - start_time
print(f"\nTraining completed in {training_time/3600:.2f} hours")

In [ ]:
@track_time
def save_model():
    model_save_path = "final_model"
    trainer.save_model(model_save_path)
    print("Model and components saved successfully")
    return model_save_path

model_save_path = save_model()
print(f"Model available at: https://huggingface.co/kasinadhsarma/vishwamai-model")

In [ ]:
@track_time
def validate_model():
    # Load all components for validation
    test_model = Transformer(ModelArgs(**model_config)).cuda()
    test_model.load_state_dict(torch.load(f"{model_save_path}/pytorch_model.bin"))
    
    # Load auxiliary components
    test_cache = DifferentiableCacheAugmentation.from_pretrained(model_save_path)
    test_memory = ReasoningMemoryTransformer.from_pretrained(model_save_path)
    test_tree = TreeOfThoughts.from_pretrained(model_save_path)
    
    test_model.eval()
    test_cache.eval()
    test_memory.eval()
    test_tree.eval()

    test_cases = [
        "What is 7 * 12?",
        "Explain quantum computing in simple terms.",
        "Write a Python function to find prime numbers."
    ]

    print("Running validation tests...")
    for test_input in test_cases:
        print(f"\nTest: {test_input}")
        # Note: You'll need to implement tokenization for the actual input
        tokens = torch.randint(0, model_config['vocab_size'], (1, 32)).cuda()
        
        with torch.inference_mode():
            start = time.time()
            output = test_model(tokens)
            end = time.time()
            
            # Apply enhancements
            enhanced_states = test_cache(output)
            memory_enhanced = test_memory(enhanced_states)
            final_output = test_tree(memory_enhanced)
            
        print(f"Generated response in {end-start:.2f}s")
        # Note: You'll need to implement detokenization for the actual output

validate_model()
print("\nPretraining and validation completed!")